In [2]:
BASE_DIRECTORY = '/content/drive/MyDrive/NLP/ENG/Jigsaw1/'
DATA_DIRECTORY = BASE_DIRECTORY + 'input/'
MODEL_DIRECTORY = BASE_DIRECTORY + 'model/'
OUTPUT_DIRECTORY = BASE_DIRECTORY + 'output/'
seed = 2390

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [3]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv(DATA_DIRECTORY + 'train.csv')
test = pd.read_csv(DATA_DIRECTORY + 'test.csv')

train_text = train.comment_text
test_text = test.comment_text
all_text = pd.concat([train_text, test_text])

In [8]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000
)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [10]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000
)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:539: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  "The parameter 'stop_words' will not be used"


In [11]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [12]:
train_features.shape, test_features.shape

((159571, 60000), (153164, 60000))

In [14]:
scores =[]
submission = pd.DataFrame.from_dict({'id': test.id})
for class_name in class_names:
  train_target = train[class_name]
  clf = LogisticRegression(C=.1, solver='sag')

  cv_score = np.mean(cross_val_score(clf, train_features, train_target, cv=3, scoring='roc_auc'))
  scores.append(cv_score)
  print(f'CV score for class {class_name} is {cv_score}')

  clf.fit(train_features, train_target)
  submission[class_name] =clf.predict_proba(test_features)[:, 1]
print(f'Total CV score is {np.mean(scores)}')
submission.to_csv(OUTPUT_DIRECTORY+'submission-lr-using-all-text.csv', index=False)

CV score for class toxic is 0.9692181774742762
CV score for class severe_toxic is 0.9875920818719472
CV score for class obscene is 0.9838684523433674
CV score for class threat is 0.9833774450514231
CV score for class insult is 0.9774236680303411
CV score for class identity_hate is 0.9739426548224612
Total CV score is 0.9792370799323028


NameError: ignored

In [15]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.982234,0.087644,0.956506,0.012203,0.864878,0.087948
1,0000247867823ef7,0.027556,0.005406,0.017158,0.002533,0.016120,0.006817
2,00013b17ad220c46,0.026810,0.005302,0.015819,0.002480,0.013871,0.005710
3,00017563c3f7919a,0.013792,0.003819,0.009990,0.002262,0.010677,0.003242
4,00017695ad8997eb,0.060085,0.004325,0.022017,0.002388,0.021510,0.004859


In [18]:
!kaggle competitions submit -c jigsaw-toxic-comment-classification-challenge -f /content/drive/MyDrive/NLP/ENG/Jigsaw1/output/submission-lr-using-all-text.csv -m "try1_tfidf_lr_using_all_text_on_tfidf"

100% 20.7M/20.7M [00:07<00:00, 2.81MB/s]
Successfully submitted to Toxic Comment Classification Challenge

In [17]:
!pip install kaggle
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle//
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json
